In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from PIL import Image
import cv2
import glob
from shutil import copyfile
import base64
from io import BytesIO
from IPython.display import HTML

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Activation
from keras.layers import Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
data_path = '..\\Data\\cohn-kanade-images'
emotion_path = '..\\Data\\Emotion'
facs_path = '..\\Data\\FACS'
landmarks_path = '..\\Data\\Landmarks'
crop_path = '..\\Data\\Crop'

all_image_files = glob.glob(data_path+'/**/*.png', recursive=True)
all_emotion_files = glob.glob(emotion_path+'/**/*.txt', recursive=True)
all_facs_files = glob.glob(facs_path+'/**/*.txt', recursive=True)
all_landmarks_files = glob.glob(landmarks_path+'/**/*.txt', recursive=True)

In [3]:
# ONLY 327 of the 593 sequences have emotion sequences
# 0-7 (i.e. 0=neutral, 1=anger, 2=contempt, 3=disgust, 4=fear, 5=happy, 6=sadness, 7=surprise)
all_emotions = {0: 'neutral', 1: 'anger', 2: 'contempt', 3: 'disgust', 
                4: 'fear', 5: 'happy', 6: 'sadness', 7: 'surprise'}
all_image_files[:5], all_emotion_files[:5], all_facs_files[:5], all_landmarks_files[:5]

([], [], [], [])

In [4]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [5]:
pd.set_option('display.max_colwidth', -1)

crop_imsize = 350

def get_thumbnail(path):
    ''' function to read image'''
    i = Image.open(path)
#     i.thumbnail((150, 150), Image.LANCZOS)
    return i

def image_base64(im):
    ''' function to encode image in base64 '''
    if isinstance(im, str):
        im = get_thumbnail(im)
    with BytesIO() as buffer:
        im.save(buffer, 'jpeg')
        return base64.b64encode(buffer.getvalue()).decode()

def image_formatter(im):
    ''' function to return html code for printing image in dataframe '''
    return '<img src="data:image/jpeg;base64,{}">'.format(image_base64(im))

def crop_image_formatter(im):
    ''' function to read cropped image from numpy array '''
    return '<img src="data:image/jpeg;base64,{}">'.format(image_base64(Image.fromarray(np.uint8(im))))

face_det = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
face_det2 = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
face_det3 = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
face_det4 = cv2.CascadeClassifier("haarcascade_frontalface_alt_tree.xml")

def detect_face(f, ld):
    ''' function to detect face, crop the image and resize it and resize the corresponding landmarks too '''
    frame = cv2.imread(f) #Open image
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
    face = face_det.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face2 = face_det2.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face3 = face_det3.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
    face4 = face_det4.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

    crop_ld = ld
    #Go over detected faces, stop at first detected face, return empty if no face.
    if len(face) == 1:
        facefeatures = face
    elif len(face2) == 1:
        facefeatures = face2
    elif len(face3) == 1:
        facefeatures = face3
    elif len(face4) == 1:
        facefeatures = face4
    else:
        facefeatures = ""
    for (x, y, w, h) in facefeatures:
        gray = gray[y:y+h, x:x+w] #Cut the frame to size
        crop_ld = np.array(list(zip(ld[:,0]-x, ld[:, 1]-y)))
    try:
        output = cv2.resize(gray, (crop_imsize, crop_imsize))
        cv2.imwrite(crop_path+'/crop_'+f.split('/')[-1], output)
    except:
        output = gray

    return output, crop_ld

def print_metrics(y_test, y_pred):
    ''' just a function from data science for printing '''
    labels = dict(all_emotions)
    del labels[0]
    
    print(labels)
    print('Accuracy: ', accuracy_score(y_test, y_pred))
    print('Classification Report: \n', classification_report(y_test, y_pred, 
                                                             target_names=[l for l in labels.values()]))
    
    conf_mat = confusion_matrix(y_test, y_pred)

    fig = plt.figure(figsize=(6,6))
    width = np.shape(conf_mat)[1]
    height = np.shape(conf_mat)[0]

    res = plt.imshow(np.array(conf_mat), cmap=plt.cm.summer, interpolation='nearest')
    for i, row in enumerate(conf_mat):
        for j, c in enumerate(row):
            if c>0:
                plt.text(j-.2, i+.1, c, fontsize=16)

    cb = fig.colorbar(res)
    plt.title('Confusion Matrix')
    _ = plt.xticks(range(len(labels)), [l for l in labels.values()], rotation=90)
    _ = plt.yticks(range(len(labels)), [l for l in labels.values()])

In [6]:
# reading all the image files, and then finding corresponding landmark, facs files and save them in a dictionary
image_dict = {}

for i_im in all_image_files:
    im_dict = {}
    im_folder = i_im.rsplit('\\', 1)[0]
    if im_folder not in image_dict.keys():
        im_files = glob.glob(im_folder+'/*.png')
        # print(i_im)
        # print(im_files)
        # print(im_folder)
        im_dict['emotion'] = im_files[-1]
        im_dict['neutral'] = im_files[0]

        im_folder_base = im_files[-1].split('\\', 3)[-1].rsplit('\\', 1)[0]
        # print(im_folder_base)
        facs_folder = facs_path + '\\' + im_folder_base
        emotion_folder = emotion_path + '\\' + im_folder_base
        landmarks_folder = landmarks_path + '\\' + im_folder_base
        # print(facs_folder)
        facs_file = glob.glob(facs_folder+'/*.txt')[0]
        emotion_file = glob.glob(emotion_folder+'/*.txt')
        neutral_landmarks_file = glob.glob(landmarks_folder+'/*.txt')[0]
        emotion_landmarks_file = glob.glob(landmarks_folder+'/*.txt')[-1]

        with open(facs_file) as f:
            data = f.read()
            im_dict['facs'] = np.array([list(map(float, i.split())) for i in data.strip().split('\n')])

        if len(emotion_file) > 0:
            emotion_file = emotion_file[0]
            with open(emotion_file) as f:
                data = f.read()
                im_dict['label'] = int(float(data))
                im_dict['label_str'] = all_emotions[im_dict['label']]

        with open(neutral_landmarks_file) as f:
            data = f.read()
            im_dict['neutral_landmarks'] = np.array([list(map(float, 
                                                     i.split())) for i in data.strip().split('\n')])

        with open(emotion_landmarks_file) as f:
            data = f.read()
            im_dict['emotion_landmarks'] = np.array([list(map(float, 
                                                     i.split())) for i in data.strip().split('\n')])

    #     print(im_dict)
        image_dict[im_folder] = (im_dict)


In [7]:
# converting that dictionary to pandas dataframe, and applying operations on certain columns
df_images = pd.DataFrame.from_dict(image_dict).T
df_images.to_csv('ImageDict.csv', index=False)

# function to store images in df column
df_images['im_emotion'] = df_images.emotion.map(lambda f: get_thumbnail(f))
df_images['im_neutral'] = df_images.neutral.map(lambda f: get_thumbnail(f))

# function to detect faces and crop images and store in df, and also shift landmarks to cropped face
# df_images['im_emotion_crop'] = df_images.emotion.map(lambda f: detect_face(f))
# df_images['im_neutral_crop'] = df_images.neutral.map(lambda f: detect_face(f))
emotion_crop_tuple = df_images.apply(lambda row: detect_face(row.emotion, 
                                                             row.emotion_landmarks), axis=1)
df_images[['im_emotion_crop', 'emotion_crop_landmarks']] = pd.DataFrame(emotion_crop_tuple.tolist(), index=emotion_crop_tuple.index)
neutral_crop_tuple = df_images.apply(lambda row: detect_face(row.neutral, 
                                                             row.neutral_landmarks), axis=1)
df_images[['im_neutral_crop', 'neutral_crop_landmarks']] = pd.DataFrame(neutral_crop_tuple.tolist(), index=neutral_crop_tuple.index)

df_images.head(3)

AttributeError: 'DataFrame' object has no attribute 'emotion'

In [ ]:
# displaying PIL.Image objects embedded in dataframe
HTML(df_images[['label', 'im_neutral', 'im_neutral_crop', 'im_emotion', 
                'im_emotion_crop']].head(5).to_html(formatters={'im_neutral': image_formatter, 
                                                                'im_emotion': image_formatter,
                                                                'im_neutral_crop': crop_image_formatter, 
                                                                'im_emotion_crop': crop_image_formatter}, escape=False))
# HTML(image_formatter(df_images[['im_neutral']].head(1)['im_neutral'][0]))
# df_images[['im_neutral']].head(1)['im_neutral'][0]

In [ ]:
ld, im = df_images.head(1)[['neutral_landmarks', 'neutral']].values[0]
implot = plt.imshow(Image.open(im))

plt.scatter([ld[i][0] for i in range(68)], [ld[i][1] for i in range(68)])
plt.show()

In [ ]:
ld, im = df_images.head(1)[['neutral_crop_landmarks', 'im_neutral_crop']].values[0]
implot = plt.imshow(im)

plt.scatter([ld[i][0] for i in range(68)], [ld[i][1] for i in range(68)])
plt.show()

In [ ]:
ld, im = df_images.head(1)[['emotion_landmarks', 'emotion']].values[0]
implot = plt.imshow(Image.open(im))

plt.scatter([ld[i][0] for i in range(68)], [ld[i][1] for i in range(68)])
plt.show()

In [ ]:
ld, im = df_images.head(1)[['emotion_crop_landmarks', 'im_emotion_crop']].values[0]
implot = plt.imshow(im)

plt.scatter([ld[i][0] for i in range(68)], [ld[i][1] for i in range(68)])
plt.show()

In [ ]:
df_cnn = df_images[~df_images['label'].isna()][['im_emotion_crop', 'label']]

In [ ]:
X, y = df_cnn['im_emotion_crop'], df_cnn['label']

In [ ]:
X_flip = X.apply(lambda x: np.fliplr(x))
plt.imshow(X_flip[0])

In [ ]:
X_total = pd.concat([X.reset_index(drop=True), X_flip.reset_index(drop=True)], axis=0).reset_index(drop=True)
X_total = np.array([i for i in X_total])
X_total = X_total.reshape(X_total.shape[0], crop_imsize, crop_imsize, 1)
y_total = to_categorical(pd.concat([y, y], axis=0).reset_index(drop=True))

X_total.shape, y_total.shape

In [ ]:
# create training and testing vars
X_train, X_test, y_train, y_test = train_test_split(X_total, y_total, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
#saving the test samples to be used later
np.save('modXtest', X_test)
np.save('modytest', y_test)

In [ ]:
X_train.shape[1], X_train.shape[2], X_train.shape[3]

In [ ]:
num_features = 64
num_labels = 8
batch_size = 64
epochs = 100
width = X_train.shape[1]
height = X_train.shape[2]
colour_channels = X_train.shape[3]

In [ ]:
# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(num_features, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(width, height, colour_channels), kernel_regularizer=l2(0.01)))
classifier.add(Conv2D(num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))
classifier.add(Dropout(0.5))


# Adding another set of layers
classifier.add(Conv2D(2*num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(Conv2D(2*num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))
classifier.add(Dropout(0.5))

# Adding a third set of layers
classifier.add(Conv2D(2*2*num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(Conv2D(2*2*num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))
classifier.add(Dropout(0.5))

# Adding a fourth set of layers
classifier.add(Conv2D(2*2*2*num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(Conv2D(2*2*2*num_features, (3, 3), padding='same', activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2, 2), strides=(2, 2)))
classifier.add(Dropout(0.5))



# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(activation = "relu",units = 2*2*2*num_features))
classifier.add(Dropout(0.4))
classifier.add(Dense(activation = "relu",units = 2*2*num_features))
classifier.add(Dropout(0.4))
classifier.add(Dense(activation = "relu",units = 2*num_features))
classifier.add(Dropout(0.5))
classifier.add(Dense(activation = "softmax",units = num_labels))

classifier.summary()

# Compiling the CNN
classifier.compile(optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7), loss = categorical_crossentropy, metrics = ['accuracy'])


In [55]:
#train the model
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, verbose=1, batch_size= batch_size, shuffle = True)

NameError: name 'true' is not defined